In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder

# 데이터 불러오기

In [ ]:
# train, test 데이터
train_data = pd.read_csv('/content/drive/MyDrive/2023 제주 특산물 가격 예측 모델/data/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/2023 제주 특산물 가격 예측 모델/data/test.csv')

display(test_data.head())
display(train_data.head())

,ID,timestamp,item,corporation,location
0,TG_A_J_20230304,2023-03-04,TG,A,J
1,TG_A_J_20230305,2023-03-05,TG,A,J
2,TG_A_J_20230306,2023-03-06,TG,A,J
3,TG_A_J_20230307,2023-03-07,TG,A,J
4,TG_A_J_20230308,2023-03-08,TG,A,J


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0


In [ ]:
# 수출입 데이터
trade_data = pd.read_csv('/content/drive/MyDrive/2023 제주 특산물 가격 예측 모델/data/international_trade.csv')
trade_data.tail(20)

,기간,품목명,수출 중량,수출 금액,수입 중량,수입 금액,무역수지
1254,2023-02,순무,4000,4,2,0,4
1255,2023-02,오이류(신선한 것이나 냉장한 것으로 한정한다),946,5,0,0,5
1256,2023-02,콩[비그나(Vigna)속ㆍ파세러스(Phaseolus)속)],331,3,2,0,3
1257,2023-02,대추야자,28,0,30850,99,-98
1258,2023-02,무화과,1319,14,104566,454,-440
1259,2023-02,파인애플,12,0,5293796,4890,-4890
1260,2023-02,구아바(guava),0,0,3,0,0
1261,2023-02,망고(mango),25,0,2432210,10374,-10374
1262,2023-02,망고스틴(mangosteen),0,0,30110,230,-230
1263,2023-02,오렌지,5944,49,3137982,6274,-6225


In [ ]:
# 제출 데이터
submission = pd.read_csv('/content/drive/MyDrive/2023 제주 특산물 가격 예측 모델/data/sample_submission.csv')

# 데이터 전처리

## train, test 데이터

In [ ]:
# timestamp 값 시계열 형태로 변환
train_data['timestamp'] = pd.to_datetime(train_data['timestamp'])
test_data['timestamp'] = pd.to_datetime(test_data['timestamp'])

type(train_data['timestamp'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [ ]:
# 날짜 데이터 분리
train_data['year'] = train_data['timestamp'].dt.year
train_data['month'] = train_data['timestamp'].dt.month
train_data['day'] = train_data['timestamp'].dt.day
train_data['weekday'] = train_data['timestamp'].dt.weekday

test_data['year'] = test_data['timestamp'].dt.year
test_data['month'] = test_data['timestamp'].dt.month
test_data['day'] = test_data['timestamp'].dt.day
test_data['weekday'] = test_data['timestamp'].dt.weekday

## 이상치 처리
- 1) supply!=0 & price==0
- 2) 명절 : 추석, 구정 -> 양력이 아닌 음력을 따르기 때문에 날짜가 고정되어 있지 않은 것들
- 3) 뜬금없이 supply==0이 되는 것들

In [ ]:
# 데이터 copy -> train_com
train_com = train_data.copy()
train_com

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,2
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,3
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,4
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,5
...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28,1
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1,2
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2,3


### 1) supply!=0 & price ==0

In [ ]:
train_com_null = train_com.loc[(train_com['supply(kg)'] != 0) & (train_com['price(원/kg)'] == 0)]
train_com_null

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday
825,TG_A_J_20210405,2021-04-05,TG,A,J,64.0,0.0,2021,4,5,0
9266,TG_D_J_20190509,2019-05-09,TG,D,J,129.0,0.0,2019,5,9,3
31199,CB_E_J_20210109,2021-01-09,CB,E,J,80.0,0.0,2021,1,9,5


#### 825열

In [ ]:
## 825열 2021-4-5 결측치 해결

target_date = pd.to_datetime(train_com_null['timestamp'].iloc[0])
start_of_week = target_date - pd.to_timedelta(3, unit='D')
end_of_week = start_of_week + pd.to_timedelta(6, unit='D')

# 해당 주에 속하는 데이터 추출
null_week_825 = train_com[(train_com['item']=='TG')&
    (train_com['corporation'] == 'A') &
   (train_com['location'] == 'J') &
   (train_com['timestamp'] >= start_of_week) &
   (train_com['timestamp'] <= end_of_week)]

train_com.loc[825, 'price(원/kg)'] = null_week_825[null_week_825['price(원/kg)']!=0]['price(원/kg)'].mean()

In [ ]:
# 비교 (0이 아닌 값으로 평균 or not)

print(null_week_825[null_week_825['price(원/kg)']!=0]['price(원/kg)'].mean())
print(null_week_825['price(원/kg)'].mean())
display(train_com.loc[822:828])

4846.2
3461.5714285714284


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday
822,TG_A_J_20210402,2021-04-02,TG,A,J,4554.5,4257.0,2021,4,2,4
823,TG_A_J_20210403,2021-04-03,TG,A,J,7075.0,3870.0,2021,4,3,5
824,TG_A_J_20210404,2021-04-04,TG,A,J,0.0,0.0,2021,4,4,6
825,TG_A_J_20210405,2021-04-05,TG,A,J,64.0,4846.2,2021,4,5,0
826,TG_A_J_20210406,2021-04-06,TG,A,J,5060.0,4910.0,2021,4,6,1
827,TG_A_J_20210407,2021-04-07,TG,A,J,2786.0,5314.0,2021,4,7,2
828,TG_A_J_20210408,2021-04-08,TG,A,J,2507.0,5880.0,2021,4,8,3


#### 9266열

In [ ]:
## 9266열 2019-5-9 결측치 해결

target_date = pd.to_datetime(train_com_null['timestamp'].iloc[1])
start_of_week = target_date - pd.to_timedelta(3, unit='D')
end_of_week = start_of_week + pd.to_timedelta(6, unit='D')

# 해당 주에 속하는 데이터 추출
null_week_9266 = train_com[(train_com['item']=='TG')&
    (train_com['corporation'] == 'D') &
   (train_com['location'] == 'J') &
   (train_com['timestamp'] >= start_of_week) &
   (train_com['timestamp'] <= end_of_week)]

train_com.loc[9266, 'price(원/kg)'] = null_week_9266[null_week_9266['price(원/kg)']!=0]['price(원/kg)'].mean()

In [ ]:
# 비교 (0이 아닌 값으로 평균 or not)

print(null_week_9266[null_week_9266['price(원/kg)']!=0]['price(원/kg)'].mean())
print(null_week_9266['price(원/kg)'].mean())
display(train_com.loc[9263:9269])

7241.666666666667
3103.5714285714284


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday
9263,TG_D_J_20190506,2019-05-06,TG,D,J,0.0,0.000000,2019,5,6,0
9264,TG_D_J_20190507,2019-05-07,TG,D,J,180.0,7500.000000,2019,5,7,1
9265,TG_D_J_20190508,2019-05-08,TG,D,J,471.0,7854.000000,2019,5,8,2
9266,TG_D_J_20190509,2019-05-09,TG,D,J,129.0,7241.666667,2019,5,9,3
9267,TG_D_J_20190510,2019-05-10,TG,D,J,2042.0,6371.000000,2019,5,10,4
9268,TG_D_J_20190511,2019-05-11,TG,D,J,0.0,0.000000,2019,5,11,5
9269,TG_D_J_20190512,2019-05-12,TG,D,J,0.0,0.000000,2019,5,12,6


#### 31199열

In [ ]:
## 31199열 2021-4-5 결측치 해결

target_date = pd.to_datetime(train_com_null['timestamp'].iloc[2])
start_of_week = target_date - pd.to_timedelta(3, unit='D')
end_of_week = start_of_week + pd.to_timedelta(6, unit='D')

# 해당 주에 속하는 데이터 추출
null_week_31199 = train_com[(train_com['item']=='CB')&
    (train_com['corporation'] == 'E') &
   (train_com['location'] == 'J') &
   (train_com['timestamp'] >= start_of_week) &
   (train_com['timestamp'] <= end_of_week)]

train_com.loc[31199, 'price(원/kg)'] = null_week_31199[null_week_31199['price(원/kg)']!=0]['price(원/kg)'].mean()

In [ ]:
# 비교 (0이 아닌 값으로 평균 or not)

print(null_week_31199[null_week_31199['price(원/kg)']!=0]['price(원/kg)'].mean())
print(null_week_31199['price(원/kg)'].mean())
display(train_com.loc[31196:31202])

980.5
560.2857142857143


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday
31196,CB_E_J_20210106,2021-01-06,CB,E,J,4940.0,752.0,2021,1,6,2
31197,CB_E_J_20210107,2021-01-07,CB,E,J,5010.0,1050.0,2021,1,7,3
31198,CB_E_J_20210108,2021-01-08,CB,E,J,4820.0,983.0,2021,1,8,4
31199,CB_E_J_20210109,2021-01-09,CB,E,J,80.0,980.5,2021,1,9,5
31200,CB_E_J_20210110,2021-01-10,CB,E,J,0.0,0.0,2021,1,10,6
31201,CB_E_J_20210111,2021-01-11,CB,E,J,0.0,0.0,2021,1,11,0
31202,CB_E_J_20210112,2021-01-12,CB,E,J,4480.0,1137.0,2021,1,12,1


In [ ]:
# LAST CHECK
train_com.loc[(train_com['supply(kg)'] != 0) & (train_com['price(원/kg)'] == 0)]

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday


## 2)명절

In [ ]:
# 휴일 패키지
!pip install holidays
!pip install pytimekr

In [ ]:
# 휴일 컬럼 설정 : holiday
import holidays
from pytimekr import pytimekr

kr_holidays = holidays.Korea(years=range(2019,2024))

for i, row in train_com.iterrows():
    date_str = row['timestamp'].strftime('%Y-%m-%d')
    is_holiday = date_str in kr_holidays
    train_com.at[i, 'holiday'] = 1 if is_holiday else 0

In [ ]:
# holiday int형으로 변환 (코랩에서 자꾸 float로 만들어버림..)
train_com[['holiday']] = train_com[['holiday']].astype(int)
train_com

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday,holiday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,1
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,2,0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,3,0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,4,0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0,2023,2,27,0,0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0,2023,2,28,1,0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0,2023,3,1,2,1
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0,2023,3,2,3,0


### **양력 날짜면서 supply==0인 1/1 데이터는 holiday에서 제외**


In [ ]:
# 1월 1일 price -> 전부 0
train_data[(train_data['month']==1)&(train_data['day']==2)]['price(원/kg)'].value_counts()

0.0    195
Name: price(원/kg), dtype: int64

In [ ]:
# CHECK
hol = train_com[(train_com['supply(kg)'] == 0) & (train_com['price(원/kg)'] == 0) & (train_com['holiday'] == 1)]

# 1/1
one = hol[(hol['month']==1)&(hol['day']==1)]

# 인덱스 리스트
one_ls = one.index.tolist()

In [ ]:
# 1/1만 holiday 값 0으로 만들기
train_com.loc[one_ls, 'holiday'] = 0
train_com

NameError: ignored

In [ ]:
# CHECK
hol_com = train_com[(train_com['supply(kg)'] == 0) & (train_com['price(원/kg)'] == 0) & (train_com['holiday'] == 1)]

# 1/1
one_com = hol_com[(hol_com['month']==1)&(hol_com['day']==1)]
one_com

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday,holiday


### 명절 및 기타 공휴일 값 보간

In [ ]:
# 해당 데이터 인덱스 리스트에 저장
nan_list = train_com[(train_com['supply(kg)']==0)&(train_com['price(원/kg)'] == 0) & (train_com['holiday'] == 1)& (train_com['weekday']!=6)].index.tolist()
nan_list

[35,
 36,
 156,
 255,
 256,
 389,
 391,
 490,
 522,
 592,
 638,
 639,
 640,
 641,
 724,
 773,
 774,
 993,
 994,
 995,
 1126,
 1127,
 1128,
 1247,
 1252,
 1348,
 1350,
 1483,
 1484,
 1558,
 1559,
 1778,
 1779,
 1912,
 1914,
 2162,
 2163,
 2164,
 2247,
 2296,
 2297,
 2516,
 2517,
 2518,
 2649,
 2650,
 2651,
 2871,
 2873,
 3006,
 3007,
 3081,
 3082,
 3301,
 3302,
 3435,
 3437,
 3536,
 3685,
 3686,
 3687,
 3819,
 3820,
 3901,
 4039,
 4040,
 4041,
 4172,
 4173,
 4174,
 4394,
 4396,
 4529,
 4530,
 4604,
 4605,
 4824,
 4825,
 4958,
 4960,
 5208,
 5209,
 5210,
 5342,
 5343,
 5562,
 5563,
 5564,
 5695,
 5696,
 5697,
 5917,
 5919,
 6052,
 6053,
 6127,
 6128,
 6217,
 6347,
 6348,
 6481,
 6483,
 6577,
 6731,
 6732,
 6733,
 6865,
 6866,
 7085,
 7086,
 7087,
 7218,
 7219,
 7220,
 7440,
 7442,
 7575,
 7576,
 7650,
 7651,
 7870,
 7871,
 8004,
 8006,
 8254,
 8255,
 8256,
 8388,
 8389,
 8608,
 8609,
 8610,
 8741,
 8742,
 8743,
 8963,
 8965,
 9098,
 9099,
 9173,
 9174,
 9263,
 9393,
 9394,
 9527,
 9529,


In [ ]:
# 3일 전, 4일 후 데이터 중 0 아닌 값으로 평균 내기
train_final = train_com.copy()

for i in nan_list:
  # 해당 주에 속하는 데이터 추출
  null_week = train_final.loc[range(i-3,i+5),:]

  null_ls = null_week.index.tolist()


  ## 7일 내내 0인 경우는 비수기로 간주하고, 그 값은 0으로 냅두기
  if all(null_week.loc[null_ls,'price(원/kg)']==0)==True:
    train_final.loc[i, 'price(원/kg)'] = 0

  else:
    train_final.loc[i, 'price(원/kg)'] = null_week[null_week['price(원/kg)']!=0]['price(원/kg)'].mean()

In [ ]:
hwas = train_final[train_final['holiday']==1].index.tolist()
print(hwas)

# 비수기에는 다 0으로 잘 돼있는지
print('비수기\n')
display(train_final.loc[32200:32250])

# 설날 값 잘 대체됐는지
print('\n설날\n')
display(train_final.loc[250:285,:])

display(train_final.loc[30:40,:])

[34, 35, 36, 59, 124, 125, 131, 156, 226, 254, 255, 256, 275, 281, 358, 388, 389, 390, 391, 425, 470, 485, 490, 522, 592, 594, 638, 639, 640, 641, 647, 724, 772, 773, 774, 790, 855, 869, 887, 957, 958, 993, 994, 995, 1006, 1007, 1012, 1014, 1089, 1126, 1127, 1128, 1155, 1163, 1220, 1223, 1247, 1252, 1322, 1347, 1348, 1349, 1350, 1371, 1377, 1378, 1454, 1481, 1482, 1483, 1484, 1520, 1557, 1558, 1559, 1582, 1647, 1648, 1654, 1679, 1749, 1777, 1778, 1779, 1798, 1804, 1881, 1911, 1912, 1913, 1914, 1948, 1993, 2008, 2013, 2045, 2115, 2117, 2161, 2162, 2163, 2164, 2170, 2247, 2295, 2296, 2297, 2313, 2378, 2392, 2410, 2480, 2481, 2516, 2517, 2518, 2529, 2530, 2535, 2537, 2612, 2649, 2650, 2651, 2678, 2686, 2743, 2746, 2770, 2775, 2845, 2870, 2871, 2872, 2873, 2894, 2900, 2901, 2977, 3004, 3005, 3006, 3007, 3043, 3080, 3081, 3082, 3105, 3170, 3171, 3177, 3202, 3272, 3300, 3301, 3302, 3321, 3327, 3404, 3434, 3435, 3436, 3437, 3471, 3516, 3531, 3536, 3568, 3638, 3640, 3684, 3685, 3686, 3687, 369

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday,holiday
32200,RD_A_J_20190806,2019-08-06,RD,A,J,0.0,0.0,2019,8,6,1,0
32201,RD_A_J_20190807,2019-08-07,RD,A,J,0.0,0.0,2019,8,7,2,0
32202,RD_A_J_20190808,2019-08-08,RD,A,J,0.0,0.0,2019,8,8,3,0
32203,RD_A_J_20190809,2019-08-09,RD,A,J,0.0,0.0,2019,8,9,4,0
32204,RD_A_J_20190810,2019-08-10,RD,A,J,0.0,0.0,2019,8,10,5,0
32205,RD_A_J_20190811,2019-08-11,RD,A,J,0.0,0.0,2019,8,11,6,0
32206,RD_A_J_20190812,2019-08-12,RD,A,J,0.0,0.0,2019,8,12,0,0
32207,RD_A_J_20190813,2019-08-13,RD,A,J,0.0,0.0,2019,8,13,1,0
32208,RD_A_J_20190814,2019-08-14,RD,A,J,0.0,0.0,2019,8,14,2,0
32209,RD_A_J_20190815,2019-08-15,RD,A,J,0.0,0.0,2019,8,15,3,1



설날



,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday,holiday
250,TG_A_J_20190908,2019-09-08,TG,A,J,0.0,0.00,2019,9,8,6,0
251,TG_A_J_20190909,2019-09-09,TG,A,J,300.0,5947.00,2019,9,9,0,0
252,TG_A_J_20190910,2019-09-10,TG,A,J,1344.0,3580.00,2019,9,10,1,0
253,TG_A_J_20190911,2019-09-11,TG,A,J,276.0,3667.00,2019,9,11,2,0
254,TG_A_J_20190912,2019-09-12,TG,A,J,57.6,3750.00,2019,9,12,3,1
255,TG_A_J_20190913,2019-09-13,TG,A,J,0.0,3259.40,2019,9,13,4,1
256,TG_A_J_20190914,2019-09-14,TG,A,J,0.0,3195.28,2019,9,14,5,1
257,TG_A_J_20190915,2019-09-15,TG,A,J,0.0,0.00,2019,9,15,6,0
258,TG_A_J_20190916,2019-09-16,TG,A,J,350.0,3400.00,2019,9,16,0,0
259,TG_A_J_20190917,2019-09-17,TG,A,J,100.0,1900.00,2019,9,17,1,0


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday,holiday
30,TG_A_J_20190131,2019-01-31,TG,A,J,15997.0,2278.0,2019,1,31,3,0
31,TG_A_J_20190201,2019-02-01,TG,A,J,36010.0,2349.0,2019,2,1,4,0
32,TG_A_J_20190202,2019-02-02,TG,A,J,8808.0,2286.0,2019,2,2,5,0
33,TG_A_J_20190203,2019-02-03,TG,A,J,0.0,0.0,2019,2,3,6,0
34,TG_A_J_20190204,2019-02-04,TG,A,J,8468.0,1831.0,2019,2,4,0,1
35,TG_A_J_20190205,2019-02-05,TG,A,J,0.0,2304.0,2019,2,5,1,1
36,TG_A_J_20190206,2019-02-06,TG,A,J,0.0,2310.0,2019,2,6,2,1
37,TG_A_J_20190207,2019-02-07,TG,A,J,0.0,0.0,2019,2,7,3,0
38,TG_A_J_20190208,2019-02-08,TG,A,J,0.0,0.0,2019,2,8,4,0
39,TG_A_J_20190209,2019-02-09,TG,A,J,2312.0,2795.0,2019,2,9,5,0


### 일요일 열 추가

In [ ]:
# 일요일 : sunday / 나머지 : non-sunday
for i in range(len(train_final)):
  if train_final.loc[i,'weekday']==6:
    train_final.loc[i,'sunday'] = 1
  else:
    train_final.loc[i,'sunday'] = 0

In [ ]:
#check
train_final.head(10)

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg),year,month,day,weekday,holiday,sunday
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0,2019,1,1,1,0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0,2019,1,2,2,0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0,2019,1,3,3,0,0.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0,2019,1,4,4,0,0.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0,2019,1,5,5,0,0.0
5,TG_A_J_20190106,2019-01-06,TG,A,J,0.0,0.0,2019,1,6,6,0,1.0
6,TG_A_J_20190107,2019-01-07,TG,A,J,44995.0,1474.0,2019,1,7,0,0,0.0
7,TG_A_J_20190108,2019-01-08,TG,A,J,26975.0,1326.0,2019,1,8,1,0,0.0
8,TG_A_J_20190109,2019-01-09,TG,A,J,29265.0,1428.0,2019,1,9,2,0,0.0
9,TG_A_J_20190110,2019-01-10,TG,A,J,21226.0,1433.0,2019,1,10,3,0,0.0


# 모델 돌리기 : AutoGluon TimeSeries Model

## autogluon 패키지 설치

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
from sklearn.preprocessing import LabelEncoder

In [ ]:
# autogluon 모듈 설치
!pip install autogluon

## test data 전처리
- sunday 표시

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/2023 제주 특산물 가격 예측 모델/data/test.csv')

In [ ]:
# timestamp 값 시계열 형태로 변환
test_data['timestamp'] = pd.to_datetime(test_data['timestamp'])

In [ ]:
# 날짜 데이터 분리
test_data['month'] = test_data['timestamp'].dt.month
test_data['day'] = test_data['timestamp'].dt.day
test_data['weekday'] = test_data['timestamp'].dt.weekday

In [ ]:
# 일요일 : 1 / 나머지 : 0
for i in range(len(test_data)):
  if test_data.loc[i,'weekday']==6:
    test_data.loc[i,'sunday'] = 1
  else:
    test_data.loc[i,'sunday'] = 0

test_data.head()

,ID,timestamp,item,corporation,location,month,day,weekday,sunday
0,TG_A_J_20230304,2023-03-04,TG,A,J,3,4,5,0.0
1,TG_A_J_20230305,2023-03-05,TG,A,J,3,5,6,1.0
2,TG_A_J_20230306,2023-03-06,TG,A,J,3,6,0,0.0
3,TG_A_J_20230307,2023-03-07,TG,A,J,3,7,1,0.0
4,TG_A_J_20230308,2023-03-08,TG,A,J,3,8,2,0.0


In [ ]:
# train 데이터랑 똑같은 형태로
test = test_data[['ID','timestamp','sunday']]
test['item_id'] = test.ID.str[0:6]

test_final = TimeSeriesDataFrame(test.drop(columns=['ID']))
test_final

<ipython-input-69-d565f2618a5c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['item_id'] = test.ID.str[0:6]


sunday
item_id timestamp         
TG_A_J  2023-03-04     0.0
        2023-03-05     1.0
        2023-03-06     0.0
        2023-03-07     0.0
        2023-03-08     0.0
...                    ...
RD_F_J  2023-03-27     0.0
        2023-03-28     0.0
        2023-03-29     0.0
        2023-03-30     0.0
        2023-03-31     0.0

[1092 rows x 1 columns]

## best quality

In [ ]:
data2 = TimeSeriesDataFrame(train_auto.drop(columns=['ID']))
predictor2 = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
    known_covariates_names=["sunday"]
)
# seed 고정
predictor2.fit(data2, presets="best_quality", time_limit=20000, random_seed=42)

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Setting presets to: best_quality
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': {'num_trials': 3,
                                'scheduler': 'local',
                                'searcher': 'auto'},
 'hyperparameters': 'best_quality',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': 300,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231119_145028/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be

  0%|          | 0/3 [00:00<?, ?it/s]

	Trained 1 models while tuning DeepAR.
	-981.6337     = Validation score (-RMSE)
	32.73   s     = Total tuning time
Hyperparameter tuning model: TemporalFusionTransformer. Tuning model for up to 29.95s of the 299.46s remaining.
	-853.9964     = Validation score (-RMSE)
	30.97   s     = Training runtime
	0.34    s     = Validation (prediction) runtime
Hyperparameter tuning model: PatchTST. Tuning model for up to 29.95s of the 299.46s remaining.
	-1074.6736    = Validation score (-RMSE)
	30.57   s     = Training runtime
	0.90    s     = Validation (prediction) runtime
Hyperparameter tuning model: DirectTabular. Tuning model for up to 29.95s of the 299.46s remaining.
	-758.4885     = Validation score (-RMSE)
	13.13   s     = Training runtime
	0.19    s     = Validation (prediction) runtime
Hyperparameter tuning model: AutoARIMA. Tuning model for up to 29.95s of the 299.46s remaining.
	Time limit exceeded... Skipping AutoARIMA.
Fitting simple weighted ensemble.
	-671.9000     = Validation 

In [ ]:
predictor2.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	85.90   s     = Training runtime
Fitting model: DeepAR/T1_FULL | Skipping fit via cloning parent ...
Fitting model: TemporalFusionTransformer_FULL | Skipping fit via cloning parent ...
Fitting model: PatchTST_FULL | Skipping fit via cloning parent ...
Fitting model: DirectTabular_FULL
	13.93   s     = Training runtime
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'RecursiveTabular_FULL', 'DeepAR/T1_FULL', 'TemporalFusionTransformer_FULL', 'PatchTS

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR/T1': 'DeepAR/T1_FULL',
 'TemporalFusionTransformer': 'TemporalFusionTransformer_FULL',
 'PatchTST': 'PatchTST_FULL',
 'DirectTabular': 'DirectTabular_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [ ]:
# seed 고정
pred2 = predictor2.predict(data2, known_covariates = test_final, random_seed=42, )

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [ ]:
# 타겟값이 음수인 데이터 확인
pred2.loc[pred2['mean']<0, 'mean']

# 모두 일요일 데이터임

item_id  timestamp 
TG_D_J   2023-03-05   -85.241223
         2023-03-19   -16.574515
CB_A_J   2023-03-05   -23.109380
         2023-03-12    -1.059460
RD_C_S   2023-03-05    -1.589544
         2023-03-12    -1.614257
         2023-03-19    -0.634104
         2023-03-26    -0.231580
BC_B_S   2023-03-05    -3.331791
         2023-03-12    -8.999215
         2023-03-19   -19.237266
         2023-03-26   -12.815303
BC_C_S   2023-03-05   -13.427334
         2023-03-12   -12.200420
         2023-03-19   -11.561214
         2023-03-26   -11.543705
Name: mean, dtype: float64

In [ ]:
# 음수 -> 0으로 대체
pred2.loc[pred2['mean']<0, 'mean'] = 0

In [ ]:
pred.head(50)

mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3381.275443  1498.804276  2150.672077  2623.816103   
        2023-03-05   968.224744 -1341.072457  -548.121927    23.195676   
        2023-03-06  3268.802682   180.106609  1233.236680  2005.884282   
        2023-03-07  3537.870106    24.001838  1238.034423  2104.615110   
        2023-03-08  3350.607171  -550.991369   802.673642  1772.891601   
        2023-03-09  3153.158283 -1114.908301   381.883577  1427.066314   
        2023-03-10  3232.289348 -1354.632481   253.897595  1384.113742   
        2023-03-11  3173.549301 -1779.389215   -42.593961  1182.597849   
        2023-03-12   184.033658 -4671.678010 -3002.910436 -1802.404568   
        2023-03-13  2790.197307 -2605.090369  -730.973320   593.299288   
        2023-03-14  3117.857158 -2608.365664  -612.350675   803.754081   
        2023-03-15  3144.570750 -2763.466239  -721.428355   717.609264   
        2023-03-16  3040.251144 -3141.211600 -1005.769439   528.760771   
        2023-03-17  3344.770729 -3088.104043  -829.590534   745.951569   
        2023-03-18  3357.045799 -3425.443482 -1051.838053   617.801202   
        2023-03-19   496.254125 -5969.468906 -3749.996775 -2152.496148   
        2023-03-20  3043.725816 -4051.136107 -1498.094590   274.137940   
        2023-03-21  3496.503709 -3730.598118 -1205.753450   556.407136   
        2023-03-22  3443.314159 -3970.853894 -1381.119876   432.138775   
        2023-03-23  3180.780933 -4395.322895 -1719.268536   144.871536   
        2023-03-24  3370.191710 -4399.938797 -1687.528259   224.375580   
        2023-03-25  3393.700831 -4768.949370 -1946.486139    92.992139   
        2023-03-26  2752.530098 -8154.561327 -5493.308961 -3578.037683   
        2023-03-27  2837.347470 -5586.023096 -2606.340294  -484.673936   
        2023-03-28  3104.948059 -5516.469388 -2528.606927  -370.237507   
        2023-03-29  3121.273673 -5843.031690 -2595.953124  -434.927663   
        2023-03-30  3224.253066 -5835.340872 -2590.456879  -381.644026   
        2023-03-31  3242.609861 -6155.480809 -2777.142546  -495.163751   
TG_A_S  2023-03-04  4048.443282  2735.142852  3192.950642  3515.434469   
        2023-03-05   954.066972  -560.629306   -36.594261   335.370670   
        2023-03-06  3917.113586  1732.915623  2506.946311  3030.564957   
        2023-03-07  3966.549891  1553.346847  2377.457946  2969.149784   
        2023-03-08  3917.231921  1204.628494  2139.374062  2796.329401   
        2023-03-09  3950.993324  1020.540524  1998.071180  2706.286757   
        2023-03-10  3960.578559   733.454727  1836.526779  2626.509637   
        2023-03-11  3948.980479   521.292365  1712.803295  2576.959309   
        2023-03-12   665.006708 -2499.795977 -1414.810528  -631.843764   
        2023-03-13  3882.300955   197.920789  1480.387884  2371.260084   
        2023-03-14  3990.438391    91.056773  1427.035853  2380.879068   
        2023-03-15  3870.550689  -178.342840  1189.762789  2188.373278   
        2023-03-16  3910.527510  -250.499516  1170.608207  2184.369083   
        2023-03-17  3900.578519  -426.682544  1067.815409  2142.387568   
        2023-03-18  3968.063410  -513.934895  1046.393599  2182.170074   
        2023-03-19   832.611504 -3373.712257 -1931.892646  -891.695239   
        2023-03-20  3872.319578  -776.880637   818.098601  1967.264349   
        2023-03-21  4110.218400  -703.775544   931.680439  2131.106246   
        2023-03-22  4000.021131 -1000.346931   767.158904  1981.977185   
        2023-03-23  4056.991029  -996.880366   768.064598  2010.158315   
        2023-03-24  4118.710000 -1152.290367   657.791367  1986.285821   
        2023-03-25  4022.507621 -1390.912078   536.410984  1873.752126   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3022.81479

In [ ]:
pred2.to_csv('/content/drive/MyDrive/2023 제주 특산물 가격 예측 모델/data/1119_submission5_300_autogluon.csv', index=False)

## (개선점) 일요일 데이터 == 0 이어야 하는데 0 아닌 경우가 더 많음 -> 엑셀에서 일요일 데이터는 0으로 대체함  